<a href="https://colab.research.google.com/github/gopalam/Computational_Geometry/blob/main/Compile_SnoCo_WQ_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is part of the WRC project.
What it does:
    Takes data files downloaded from SnoCo Water Quality Viewer, and compiles them into a master file, one for each parameter.


# libraries to import
**pandas** is a library that lets you work with spreadsheets.It is the Python equivalent of Excel.

**os** stands for operating system, this library allows us to do things outside of python using tools that are inbuilt in your computer

***glob*** is a library that searches for files

**re** is a library that helps sort filenames in an order

In [4]:
import pandas as pd
import glob # this is a library that searches for files
import re # this helps sorting filenames in an order
import os

lets tell the code where our files are located. Click on the folder shaped icon on the left side of this page, and then click on the folder with the Google drive Icon on it, and say yes to this code making changes to drive.

Go to the browser page of your Google Drive account. Right click on the folder that you were shared "WRC remote sensing project_2023.2024", go to "share" and to "add shortcut", this lets you add a shortcut to this folder into the folders on drive that you own. So choose any location in drive where you want this folder to appear. Once that is done, you can then come back to this window, and navigate to that folder on drive (if the folder does not appear, click on the "refresh folder" icon), right click on that folder, and click on "copy path", this will copy a link to the folder, that you can add to the working directory variable below


In [1]:

working_dir=r'/content/drive/MyDrive/DOH_Sentinel/WRC remote sensing project_2023.2024'
working_dir=working_dir+"//**//"
file_types='*.csv' # what kind files are we looking for, * stands for wildcard.

print(working_dir)

/content/drive/MyDrive/DOH_Sentinel/WRC remote sensing project_2023.2024//**//


In [2]:
def file_list(root_folder,file_types):
    def natural_sort(l):
        convert = lambda text: int(text) if text.isdigit() else text.lower()
        alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
        return sorted(l, key=alphanum_key)
    files=[]
    root_path=root_folder+file_types
    for file in glob.glob(root_path,recursive=True):
        files.append(file)
        files=natural_sort(files)
    files_path=files.copy()
    for ind in range(0,len(files_path)):
        pth=files_path[ind]
        tif_file=(os.path.basename(pth))
        mod_pth=pth.replace(tif_file,'')
        files_path[ind]=mod_pth
        files[ind]=tif_file
    return files, files_path

Now run the code that asks the code to compile a list of files in the folder

In [11]:
files,file_paths=file_list(working_dir,file_types)
print(files)

['Armstrong_Chlorophylla_all_2023-12-28_13-38-04-08-00.csv', 'Armstrong_LakeLevel_all_2023-12-28_13-38-46-08-00.csv', 'Armstrong_Phaeophytin_all_2023-12-28_13-39-10-08-00.csv', 'Armstrong_Phycocyanin_all_2023-12-28_13-39-29-08-00.csv', 'Armstrong_Secchi_all_2023-12-28_13-39-48-08-00.csv', 'Armstrong_Temperature-air_all_2023-12-28_13-40-05-08-00.csv', 'Armstrong_Temperature-water_all_2023-12-28_13-40-19-08-00.csv', 'Armstrong_TotalPersulfateNitrogen_all_2023-12-28_13-40-36-08-00.csv', 'Armstrong_TotalPhosphorus_all_2023-12-28_13-41-08-08-00.csv', 'Armstrong_TrueColor_all_2023-12-28_13-41-41-08-00.csv', 'Bosworth_Chlorophylla_all_2023-12-28_14-53-47-08-00.csv', 'Bosworth_Conductivity_all_2023-12-28_14-54-05-08-00.csv', 'Bosworth_LakeLevel_all_2023-12-28_14-54-25-08-00.csv', 'Bosworth_Phaeophytin_all_2023-12-28_14-54-49-08-00.csv', 'Bosworth_Phycocyanin_all_2023-12-28_14-55-13-08-00.csv', 'Bosworth_Secchi_all_2023-12-28_14-55-33-08-00.csv', 'Bosworth_Temperature-air_all_2023-12-28_14-55-5

Once we run the above line of code, we get two variables each called a list, that contain file names and their locations. The lines below once executed will cycle through each file, read each one in using the spreadsheet library and pick up the variables we are looking for, and copy them all into one master sheet cycling through the files

In [14]:
df_all=pd.DataFrame()

for file,fpath in zip(files,file_paths):
    #add the file to the file path, so we can reach the corrrect folder where the file is located.
    fname=fpath+file
    # print(fname)
    # lets read it into Python in two different ways
    # one as a spreadsheet
    df=pd.read_csv(fname,encoding='windows-1252',skiprows=[0,1,2,3,4,5,6,7,8,9,10,11])
    # another as a dictionary
    out_data={}
    with open(fname,encoding='windows-1252')as f:
        ind=0
        for line in f:
            val=line.strip()
            out_data[ind]=val
            ind=ind+1
   # lets cycle through the data, identify the lat and long and the parameter name, units and lake name
    for key in out_data:
        line=out_data[key]
        #split the line by a comma, but first remove the hash in each line if present
        line=line.replace("#",'')
        # print(line)
        varlist=line.split(",")
        if(varlist[0]=='ts_name'):
            varname=varlist[1]
            # print(varname)
        if(varlist[0]=='station_name'):
            lake_name=varlist[1]
            # print(lake_name)
        if(varlist[0]=='station_latitude'):
            lat=varlist[1]
        if(varlist[0]=='station_longitude'):
            lon=varlist[1]
        if(varlist[0]=='ts_unitsymbol'):
            units=varlist[1]
        # now we will identify the parameter name, and add a column with that name as well as units
    df['parameter']=varname
    df['latitude']=lat
    df['longitude']=lon
    df['units']=units
    df['lake_name']=lake_name
    df=df.rename(columns={varname:'value'})
    df=df[['lake_name','parameter','latitude', 'longitude','timestamp', 'value', 'units', 'method', 'wqremarkstd', 'depth', 'samplenumber']]
    df_all=pd.concat([df_all,df])
        # we will append these variables to the table.


In [15]:
# lets look at a part of the compiled master file
df_all.head(100)

,lake_name,parameter,latitude,longitude,timestamp,value,units,method,wqremarkstd,depth,samplenumber
0,Armstrong,Chlorophyll a,48.22526,-122.12319,2015-06-13 13:30:00,1.600,µg/l,SM10200H,NaN,3.28,1
1,Armstrong,Chlorophyll a,48.22526,-122.12319,2015-07-11 15:30:00,13.000,µg/l,SM10200H,NaN,3.28,1
2,Armstrong,Chlorophyll a,48.22526,-122.12319,2015-08-08 13:30:00,4.300,µg/l,SM10200H,NaN,3.28,1
3,Armstrong,Chlorophyll a,48.22526,-122.12319,2015-09-12 13:00:00,4.000,µg/l,SM10200H,NaN,3.28,1
4,Armstrong,Chlorophyll a,48.22526,-122.12319,2016-06-12 13:15:00,15.000,µg/l,SM10200H,NaN,3.28,1
...,...,...,...,...,...,...,...,...,...,...,...
14,Armstrong,Secchi,48.22526,-122.12319,2016-06-27 13:00:00,2.150,m,Secchi-L,NaN,NaN,1
15,Armstrong,Secchi,48.22526,-122.12319,2016-07-09 14:30:00,1.950,m,Secchi-L,NaN,NaN,2
16,Armstrong,Secchi,48.22526,-122.12319,2016-07-23 15:00:00,2.150,m,Secchi-L,NaN,NaN,1
17,Armstrong,Secchi,48.22526,-122.12319,2016-08-13 14:00:00,2.125,m,Secchi-L,NaN,NaN,2


Now lets look into the number of unique parameters for which values where collected, and how many values for each.

In [17]:
vrs=df_all['parameter'].unique().tolist()
print('unique parameters:',vrs)
vals=df_all.value_counts('parameter')
print('----------------------------------------------')
print(vals)


unique parameter ['Chlorophyll a', 'Lake Level', 'Phaeophytin', 'Phycocyanin', 'Secchi', 'Temperature - air', 'Temperature -water', 'Total Persulfate Nitrogen', 'Total Phosphorus', 'True Color', 'Conductivity', 'Soluble Reactive Phosphorus', 'Lake Depth']
----------------------------------------------
parameter
Temperature -water             19251
Temperature - air               3210
Total Phosphorus                3063
Secchi                          3058
Conductivity                    2042
Chlorophyll a                   1565
Total Persulfate Nitrogen       1376
Phaeophytin                     1085
Phycocyanin                     1046
Lake Level                       630
Lake Depth                       534
Soluble Reactive Phosphorus      525
True Color                       282
dtype: int64


This code takes that compiled spreadsheet that is in memory and saves it as csv files, one for each parameter.

In [ ]:

vrs=df_all['parameter'].unique().tolist()
store_dir=r'/content/drive/MyDrive/DOH_Sentinel/WRC remote sensing project_2023.2024\\'
for vr in vrs:
    df=df_all[df_all['parameter']==vr]
    fname='SnoCo_compiled_'+vr+'.csv'
    df.to_csv(store_dir+fname)

